# First steps with Gym-dssat

In this tutorial, we will show how to simply interact with the gym-dssat environment.

In [ ]:
import gym
import gym_dssat_pdi
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

Next, we instantiate the environment. Here, we will use the fertilization mode...

In [ ]:
# create env
env_args = {
  'run_dssat_location': '/opt/dssat_pdi/run_dssat',
  'mode': 'fertilization',
  'seed': 123,
  'random_weather': True,
}

env = gym.make('GymDssatPdi-v0', **env_args)

Now, we can interact with our environment. We will do this with two hardcoded policies:
- A null policy
- An expert policy

and display results of their evaluation.

To facilitate this, we define the following helper functions:

In [ ]:
def null_policy(obs):
    '''
    Do not apply any fertilizer
    '''
    return { "anfer": 0}

def expert_policy(obs):
    '''
    Return fertilization amount based on internal map
    and day after planting feature in observation.
    '''
    fertilization_map = {
        40: 27,
        45: 35,
        80: 54,
    }

    amount = fertilization_map.get(obs['dap'], 0)
    
    return { "anfer": amount }


def evaluate(policy, n_episodes=10):
    returns = [None] * n_episodes

    for episode in range(n_episodes):
        done = False
        obs = env.reset()
        ep_return = 0

        while not done:
            action = policy(obs)
            obs, reward, done, _ = env.step(action)
            if reward is not None:
                ep_return += reward

        # sanity check REMOVE THIS IN FINAL TUTORIAL
        assert ep_return == sum(env._history['reward']), "Computed return doesn't match internal history of rewards"

        returns[episode] = ep_return

    return returns

def plot_results(labels, returns):
    data_dict = {}
    for label, data in zip(labels, returns):
        data_dict[label] = data
    df = pd.DataFrame(data_dict)
    
    ax = sns.boxplot(data=df)
    ax.set_xlabel("policy")
    ax.set_ylabel("evaluation output")
    plt.savefig('results.pdf')
    plt.show()

Evaluate null policy

In [ ]:
null_returns = evaluate(null_policy)

Evaluate expert policy

In [ ]:
expert_returns = evaluate(expert_policy)

Finally, we display the results in the following plot. A copy is also saved as `results.pdf` in the current working directory

In [ ]:
labels = ['null', 'expert']
returns = [null_returns, expert_returns]
plot_results(labels, returns)

And, there you have it! Now, you can go ahead and keep playing with gym-dssat.
<br/><br/>
When done, close the environment

In [ ]:
env.close()